# LDA Topic Model of FEMALE Antivax+AntiAntivax Comments on Facebook

In [1]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import nltk

import os
from time import time
from collections import defaultdict

import numpy as np
import pandas as pd

import gensim
from gensim import corpora, utils
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers.dtmmodel import DtmModel

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib
import matplotlib.pyplot as plt

from dateutil import parser
import datetime

import random

%matplotlib inline

In [2]:
import csv

#with open('REDDIT_DATASET_ONE_YEAR_COMMENTS.csv', 'r') as f:
#  reader = csv.reader(f)
#  your_list = list(reader)

# with open('REDDIT_DATASET_ONE_YEAR_COMMENTS_FEMALE_RELATED_COMMENTS.csv', 'r') as f:
#   reader = csv.reader(f)
#   your_list = list(reader)

with open('./data/combined_female_data.csv', 'r') as f:
   reader = csv.reader(f)
   your_list = list(reader)

# reader = pd.read_csv('./data/combined_male_data.csv', sep=',', encoding='latin-1')
# your_list = reader['x'].tolist()
    
your_list.pop(0) # remove the first element which is accidentally put there by R after exporting

['x']

In [3]:
print(your_list[:5]) # first 5 elements
# print(reader[:5]) # first 5 elements

[["I'm confused (it happens a lot with these threads) if rabies is psychosomatic, what virus strands is the vitamin c attaching to?"], ['I just got into a heated argument with an anti-vaxxer. I left him with this:\n"So-called antivaccinationists have claimed over the years that vaccines do not work, despite the overwhelming evidence that they do. They often spread misinformation about vaccine, such as the notion that vaccines weaken the immune system, when in fact they work by strengthening the immune response against the target infection."\n(https://www.sciencebasedmedicine.org/reference/vaccines-and-autism/)'], ['Even TB testing???? Does this woman realize exactly how TB is spread or how it kills? Does she understand that those with TB HAVE to take their medication or be isolated and/or imprisoned?\n\nIs she really that stupid?\n\nWait.... please don\'t answer that. I can already glean the answer from her "contain aborted fetal cell tissue" line...'], ['This is exactly how the zombie

In [4]:
len(your_list)

181258

In [5]:
import itertools
merged_list = list(itertools.chain(*your_list))

In [6]:
# import random
# merged_list = merged_list[1:10000] # we create a small toy dataset to work with 
# merged_list = random.sample(merged_list, 100000)
# print(merged_list[:5]) # view first 5 elements

In [7]:
len(merged_list)

181258

In [8]:
comments_text = [comment for comment in merged_list]
comments_text_original = [comment for comment in merged_list]

In [9]:
# do word stemming?
from stemming.porter2 import stem

comments_text = [[stem(word) for word in sentence.split(" ")] for sentence in comments_text]
comments_text = [" ".join(sentence) for sentence in comments_text]

In [10]:
# print(comments_text[:5])

## Generate topic model

In this section we generate the topic model. There are some particular parameters that have an important role in shaping the outcome of the model. Changing these will affect the output in various ways. 

I have tested quite a few different combinations and the current output seems to give reasonable results (although we can certainly do better).

num_topics = 30 ... we specify that we want to generate 30 topics from our data
word stemming ... we use a word stemmer to reduce some words to their roots (e.g., walked and walking becomes "walk")
n-grams ... we specify that we want single terms (e.g. "walked"), and also find bi-grams or 2-grams (e.g., "she walked")
stop words ... we remove common English words that are not useful for analysis (e.g. "the", "and")
max_df ... we ignore terms that have a document frequency strictly higher than the given threshold
min_df ... ignore terms that have a document frequency strictly lower than the given threshold


In [11]:
# for good overview of these values see: http://stackoverflow.com/a/35615151/2589495
max_df = 0.05 # ignore terms that have a document frequency strictly higher than x% of docs (or occurs in > n docs if integer value)
min_df = 2 # ignore terms that have a document frequency strictly less than x% of documents (or occurs in < n documents if integer value)
# Some good values found so far...
# max_df = 0.05 (smaller in same order of magnititude is also good e.g. 0.03)
# min_df = 2
# k topics = 50
# passes = 3

# bi-grams:
# tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,ngram_range=(1,2),stop_words='english',analyzer='word')
# unigrams 
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,stop_words='english')
tf = tf_vectorizer.fit_transform(comments_text)

In [12]:
# dictionary.filter_tokens(bad_ids=low_value_words)
# corpus = [dictionary.doc2bow(doc) for doc in texts]
corpus = gensim.matutils.Sparse2Corpus(tf.T)
n_terms, n_docs = corpus.sparse.shape
id2word = {i:word for i, word in enumerate(tf_vectorizer.get_feature_names())}
dictionary = Dictionary.from_corpus(corpus, id2word=id2word)

In [13]:
# generate LDA model
number_topics = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=number_topics, id2word = dictionary, passes=100,update_every=1, chunksize=10000)

In [14]:
# ldamodel.print_topics(-1)

## Visualisation of topics (by term/word)

This is an interactive visualisation of the topic model. The right-hand side shows which terms belong to each topic and how important they are for that topic (in order of importance). 

The left-hand Intertopic Distance Map gives an indication of how closely related the topics are (projected into a two-dimensional space). It may or may not be that helpful analytically, but is included anyway. 

In [15]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis, 'lda_model_FEMALE_antivax_and_antiantivax.html')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [16]:
pyLDAvis.display(vis)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.390295. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [17]:
docTopicProbMat = ldamodel[corpus]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.397879. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [18]:
docTopicProbMat_list = list(docTopicProbMat)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.398838. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [19]:
# print(docTopicProbMat_list[7]) # get the topic probabilities for a document
# print(comments_text[7]) # and the document text

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.400057. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [33]:
comment_probabilities = []
for doc in docTopicProbMat_list:
    docTopic = 999 # reset the topic
    bestMatch = 0 # reset the best match
    possibleMatch = 0 # reset the possible match
    for topic in doc:
        possibleMatch = topic[1]
        if possibleMatch > bestMatch:
                bestMatch = topic[1]
                if bestMatch > 0.5: # minimum threshold for comment topic relatedness
                    docTopic = topic[0]
                # print("bestMatch is now: ",bestMatch)
                # print("docTopic is now: ",docTopic)
    # print("best topic was: ",docTopic)
    comment_probabilities.append(docTopic)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:41:41.798914. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [21]:
# comment_probabilities[0:10]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.402277. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [34]:
comment_probabilities = [probs + 1 for probs in comment_probabilities] # now when we ask for topic n we actually get topic n (not topic n-1)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:41:53.523022. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [23]:
# print(comments_text[8])
# comment_probabilities[8]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.404371. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [35]:
comments_and_topics = pd.DataFrame(
    {'original_comment_text': comments_text_original,
     'comment_text': comments_text,
     'topic': comment_probabilities
    })

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:41:56.954543. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [25]:
# comments_and_topics

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-17 22:07:45.406698. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [36]:
# a function to get n random example comments assigned to a given topic k
def getTopicExamples(topic_id,num_examples):
    topic_comments = list(comments_and_topics['original_comment_text'][comments_and_topics.topic == topic_id])
    topic_comments = random.sample(topic_comments, num_examples)
    return(topic_comments)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:42:11.690500. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [37]:
## DEBUG -- CHECK HOW MANY COMMENTS IN EACH TOPIC
for i in range(1, number_topics):
    topic_comments = list(comments_and_topics['original_comment_text'][comments_and_topics.topic == i])
    print(len(topic_comments))

743
213
588
1113
3501
97
124
192
86
2226
57
680
417
774
377
487
518
894
1075
896
761
116
1061
3087
538
86
3875
2784
723


/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:42:18.794047. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Extract 10 comment examples for each topic

In this section we extract 50 (randomly selected) text comments that belong to each topic. 

This provides some examples of comments that belong to each topic. The topics are ordered from 1 to k (where k is the number of topics we specified for the model). 

You can use these comment examples to help make sense of the topic terms in the above visualisation, in other words "what is going on" for each topic.

In [32]:
print(k)

11


/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:40:57.693569. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [38]:
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    print("\n____________________ TOPIC",k,"______________________________\n")
    print(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')


____________________ TOPIC 1 ______________________________

They do it because some doctor told them to do it, and they believed him(her) w/o checking out all the facts and information for themselves.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
think carefully and critically..once you do this you can't take it back..
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
How can I trust a doctor with my health if his name contains "cola"?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
He's part of the paradigm shift. He HAS gone to the doctor but only to hear the rant and sign the waiver.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Maybe she should take her kid to the chiropractor ER then.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Omg, nine? This is why registered nurses need to be working in doctors offices not medical assistants.  Medical assistants should not be calling themselves nurses, because they do not have the same traini

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:42:38.066428. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [40]:
f1=open('./example_comments_numComments50_numTopics30_FEMALE_DATA.txt', 'w+')
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    topic_example_comments = "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n".join(item for item in topic_example_comments)
    topicheader = "\n____________________ TOPIC",k,"______________________________\n"
    f1.write(str(topicheader))
    # f1.write(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')
    f1.write("\n"+topic_example_comments+"\n")

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-18 09:45:31.746928. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
# theta, _ = ldamodel.inference(corpus) # we can extract the thetas directly from the corpus

In [ ]:
# theta[8] # columns are indexed from 0, so topic n is topic n+1 (e.g. topic22 is topic23)

## Word2vec analysis of text

In this section we do some exploratory text analysis using the novel word2vec algorithm.

This is very experimental... just some computational doodling.

In [ ]:
sentences = [comment for comment in merged_list]
sentences = [[stem(word) for word in sentence.split(" ")] for sentence in sentences]

In [ ]:
# import nltk
# nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
filtered_words = [word for word in sentences if word not in stopwords.words('english')]

In [ ]:
# print(filtered_words[2]) # sanity check the data first

In [ ]:
# convert to lowercase
filtered_lower_words = []
for doc in filtered_words:
    filtered_lower_words.append([word.lower() for word in doc])

In [ ]:
# remove punctuation
import string
filtered_lower_words_nopunct = []
for doc in filtered_lower_words:
    filtered_lower_words_nopunct.append([''.join(c for c in s if c not in string.punctuation) for s in doc])

In [ ]:
# remove empty words due to previous steps 
filtered_lower_words_nopunct_nospaces = []
for doc in filtered_lower_words_nopunct:
    filtered_lower_words_nopunct_nospaces.append([s for s in doc if s])

In [ ]:
# print(filtered_lower_words_nopunct_nospaces[2]) # sanity check the data quickly

In [ ]:
from gensim.models import Word2Vec
min_count = 50
size = 120
window = 7
workers = 6
 
model = Word2Vec(filtered_lower_words_nopunct_nospaces, min_count=min_count, size=size, window=window)


Let's try the classic word2vec example:

King - man + woman = ?

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

Using this kind of analysis we can ask other questions more specific to the paper:
    
vaccine - man + woman = ?

(What is 'vaccine' without taking into account 'men', but taking into account 'women'?)

In [ ]:
model.wv.most_similar(positive=['women', 'vaccine'], negative=['men'])

And we can also do the converse for men:

vaccine - woman + man = ?

(What is 'vaccine' without taking into account women, but taking into account men?)

In [ ]:
model.wv.most_similar(positive=['men', 'vaccine'], negative=['women'])

In [ ]:
model.most_similar("angry")

In [ ]:
model.most_similar("autism")

In [ ]:
model.most_similar(["choice"])

We can also see which word within a set of words doesn't belong:

In [ ]:
model.wv.doesnt_match("choice science vaccine".split())
